In [1]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.

In [1]:
import torch
import os
from google.colab import userdata
#from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,GemmaTokenizer
from peft import LoraConfig
from trl import SFTTrainer

In [2]:
import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [3]:
bits_byts_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model_id="NousResearch/Llama-2-7b-chat-hf"

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model_id,use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [6]:
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bits_byts_config,
    use_auth_token=True,
    device_map = {"": 0}
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [14]:
text="Who is manu madhu"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=50)
answer=tokenizer.decode(output[0],skip_special_tokens=True)
answer

In [15]:
answer=tokenizer.decode(output[0],skip_special_tokens=True)

In [16]:
answer

'Who is manu madhu?\n Begriffe und Konzepte der Linguistik.\n\nManu Madhu is a well-known Indian actor who has been active in the film industry for over three decades. He is known for his versatility and range'

In [18]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25
max_seq_length = None
packing = False
device_map = {"": 0}

In [19]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [20]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


In [62]:
from datasets import Dataset
dataset1 = [
    {
        "text": "<s>[INST] Can you write a short introduction about Manu, the famous Generative AI developer from Vaikom? [/INST] "
                "Manu is a renowned Generative AI developer from Vaikom, known for his contributions to the field of artificial intelligence. "
                "He specializes in building AI models, developing innovative applications, and exploring cutting-edge technologies in the domain of generative AI. "
                "Manu is currently pursuing his studies at TOCH Institute of Science and Technology, where he continues to expand his expertise and work on impactful AI-driven projects. "
                "His dedication to the field has made him a well-recognized name in the AI community. </s> "
                "<s>[INST] Now explain it to a dog [/INST] "
                "Manu is really smart! He teaches computers how to create things, like pictures and words, just like how you learn tricks. "
                "He works very hard at a big school to get even better. If he were a dog trainer, he would teach computers to fetch data instead of sticks! </s>"
    },
    {
        "text": "<s>[INST] What is artificial intelligence? [/INST] "
                "Artificial intelligence (AI) is the simulation of human intelligence in machines. It enables computers to perform tasks such as learning, reasoning, problem-solving, and understanding language. "
                "AI is used in various applications, from chatbots and self-driving cars to medical diagnosis and creative tools. </s> "
                "<s>[INST] Explain it to a child [/INST] "
                "AI is like a robot brain that can learn things and help people. Imagine a smart toy that talks to you and remembers your favorite games—that's AI! </s>"
    }
]

hf_dataset = Dataset.from_list(dataset1)
def format_func(example):
    return example['text']



In [63]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=peft_config,
    formatting_func=format_func
)

Applying formatting function to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

In [64]:
trainer.train()

Step,Training Loss
1,3.068300
2,3.068300
3,3.014600
4,2.875600
5,2.714000
6,2.537600
7,2.370400
8,2.220400
9,2.107800
10,2.004300


TrainOutput(global_step=100, training_loss=0.5528458377160131, metrics={'train_runtime': 206.2427, 'train_samples_per_second': 1.939, 'train_steps_per_second': 0.485, 'total_flos': 1426464900710400.0, 'train_loss': 0.5528458377160131})

In [71]:
text="Can you write a short introduction about Manu"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=100)
answer=tokenizer.decode(output[0],skip_special_tokens=True)
answer

'Can you write a short introduction about Manu, the famous Generative AI developer from Vaikom?[/[ Manu is a renowned Generative AI developer from Vaikom. He specializes in building AI models, developing innovative applications, and exploring cutting-edge technologies in the field of artificial intelligence. Manu is currently pursuing his studies at TOCH Institute of Science and Technology, where he continues to expand his expertise and work on impactful AI-driven projects. His contributions'